自动文摘的目的是通过对原文本进行压缩、提炼，为用户供简明扼要的文字描述。自动文摘是一个信息压缩过程，将输入的一篇或多篇文档内容总结为一段简要描述，该过程不可避免有信息损失，但是要求保留尽可能多的重要信息，自动文摘也是自然语言生成领域中一个重要任务。
下面我们以文本摘要任务为例，展示孟子预训练模型在下游任务上微调的流程，整体流程可以分为4部分：

- 数据加载
- 数据预处理
- 模型训练
- 模型推理

下面我们以中文科学文献数据（CSL）文本摘要数据为例进行演示，数据下载地址：https://github.com/CLUEbenchmark/CLGE

1. 数据加载

CSL数据以json的形式存储，通过如下方式可以将数据加载进内存。

依赖环境
- transformers
- py-rouge

In [54]:
import json
import time
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from tqdm import tqdm
import torch

In [45]:
def read_json(input_file: str) -> list:
    '''
    读取json文件，每行是一个json字段

    Args:
        input_file:文件名

    Returns:
        lines
    '''
    with open(input_file, 'r') as f:
        lines = f.readlines()
    return list(map(json.loads, tqdm(lines, desc='Reading...')))

train = read_json("csl/v1/train.json") 
dev = read_json("csl/v1/dev.json") 

Reading...: 100%|██████████| 500/500 [00:00<00:00, 129166.79it/s]


下面展示数据集的具体信息：

In [53]:
print('训练集大小：%d个训练样本'%(len(train)))
print('每个训练样本的原始格式如下：\n',train[1])

训练集大小：2500个样本
每个训练样本的原始格式如下：
 {'id': 2, 'title': '一种基于混合模型的实时虚拟人服装动画方法', 'abst': '实时服装动画生成技术能够为三维虚拟角色实时地生成逼真的服装动态效果,在游戏娱乐、虚拟服装设计展示等领域有着广泛的应用前景.其难点在于如何建立服装动画计算模型,在实时计算的前提下获得最佳的服装动画生成效果.在对服装模型与人体模型在运动过程中发生的位置冲突(collision,也称碰撞)进行分析的基础上,研究并提出了一种基于混合模型的实时虚拟人服装动画计算模型.首先,根据服装动画样本数据中服装与人体发生位置冲突的信息,对服装与人体的运动相关性进行分析;在此基础上,提出并实现一种新的混合策略,将具有较好服装动态模拟效果的动力学计算模型与具有较高计算效率的几何变形方法进行混合,建立支持实时计算且效率可动态控制的服装动画计算模型.实验结果表明,该计算模型能够实时地生成具有较好视觉逼真性的服装动画.'}


可以看出每条原始数据包含3个字段，分别是id，title，abst，其中id是唯一标识，abst是文本摘要任务的输入，title是文本摘要任务的输出。

2. 数据预处理

数据预处理的目的是将原始数据处理为模型可以接受的输入形式，相当于在原始数据和模型输入之间建立管道。
模型输入，可接受的字段为input_ids、labels，其中input_ids为输入文本的tokenized表示，可以直接通过transformers提供的Tokenizer进行转换；labels为模型期望输出文本的tokenized表示。
通过定义DataCollatorForSeq2Seq数据预处理类，将其传递给data_collator完成上述流程，数据预处理代码如下：

In [56]:
model_path = "Langboat/mengzi-t5-base" # huggingface下载模型

##### 加载预训练模型，包括分词器tokenizer和model。

In [7]:
Mengzi_tokenizer = T5Tokenizer.from_pretrained(model_path)

In [8]:
Mengzi_model = T5ForConditionalGeneration.from_pretrained(model_path)

In [18]:
class Seq2SeqDataset:
    def __init__(self, data):
        self.datas = data

    def __len__(self):
        return len(self.datas)

    def __getitem__(self, index):
        return self.datas[index]

class DataCollatorForSeq2Seq:
    def __init__(self, tokenizer, padding: bool = True, max_length: int = 512):
        self.tokenizer = tokenizer
        #self.model = model
        self.padding = padding
        self.max_length = max_length

    def __call__(self, batch):
        features = self.collator_fn(batch)
        return features


    def preprocess(self, item):
        source = item["abst"]
        target = item["title"]
        return source, target

    def collator_fn(self, batch):
        results = map(self.preprocess, batch)
        inputs, targets = zip(*results)

        input_tensor = self.tokenizer(inputs,
                                      truncation=True,
                                      padding=True,
                                      max_length=self.max_length,
                                      return_tensors="pt",
                                      )

        target_tensor = self.tokenizer(targets,
                                       truncation=True,
                                       padding=True,
                                       max_length=self.max_length,
                                       return_tensors="pt",
                                       )

        input_tensor["labels"] = target_tensor["input_ids"]

        if "token_type_ids" in input_tensor:
            del input_tensor["token_type_ids"]
        return input_tensor

In [19]:
trainset = Seq2SeqDataset(train)
devset = Seq2SeqDataset(dev)

In [20]:
collator = DataCollatorForSeq2Seq(Mengzi_tokenizer)

3. 模型训练

训练模型前需要指定模型训练的超参数，包括训练的轮数、学习率和学习率管理策略等等：可以通过实例化TrainingArguments类来，并将其传递给Trainer来传入这些超参数。
然后通过huggingface定义的trainer.train()方法来进行训练。
训练完成后通过trainer.save_model()方法来保存最佳模型。

In [22]:
output_dir = "test" # 模型checkpoint的保存目录
training_args = TrainingArguments(
        num_train_epochs=3,
        per_device_train_batch_size=8,
        logging_steps=10,
        #fp16=True,
        evaluation_strategy="steps",
        eval_steps=100,
        load_best_model_at_end=True,
        learning_rate=1e-5,
        #warmup_steps=100,
        output_dir="test",
        save_total_limit=5,
        lr_scheduler_type='constant',
        gradient_accumulation_steps=1,
        dataloader_num_workers=4)

In [23]:
print('Tarining Arguments ...')
print(training_args)

trainer = Trainer(
    tokenizer=Mengzi_tokenizer,
    model=Mengzi_model,
    args=training_args,
    data_collator=collator,
    train_dataset=trainset,
    eval_dataset=devset
)

trainer.train()
trainer.save_model("test/best") # 保存最好的模型

Tarining Arguments ...
TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=4,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=100,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=False,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model_at_end=True,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=test/runs/Nov01_11-42-16_JX-ZY-GPU12,
logging_first_step=False,
logging_steps=10,
logging_strategy=IntervalStrategy.STEPS,
lr_scheduler_type=SchedulerType.CONSTANT,
max_grad_norm=1.0,
max_steps=

***** Running training *****
  Num examples = 2500
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 939
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: egyang (use `wandb login --relogin` to force relogin)


Step,Training Loss,Validation Loss
100,1.781900,1.654125
200,1.669000,1.454569
300,1.399100,1.426801
400,1.294300,1.377249
500,1.362800,1.363519
600,1.497200,1.342906
700,1.297500,1.321924
800,1.294700,1.321687
900,1.111900,1.306872


***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test/checkpoint-100
Configuration saved in test/checkpoint-100/config.json
Model weights saved in test/checkpoint-100/pytorch_model.bin
tokenizer config file saved in test/checkpoint-100/tokenizer_config.json
Special tokens file saved in test/checkpoint-100/special_tokens_map.json
Copy vocab file to test/checkpoint-100/spiece.model
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test/checkpoint-200
Configuration saved in test/checkpoint-200/config.json
Model weights saved in test/checkpoint-200/pytorch_model.bin
tokenizer config file saved in test/checkpoint-200/tokenizer_config.json
Special tokens file saved in test/checkpoint-200/special_tokens_map.json
Copy vocab file to test/checkpoint-200/spiece.model
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test/checkpoint-300
Configuration saved in t

4. 模型推理

最佳模型保存在了"test/best"位置，我们可以加载最佳模型并利用其进行摘要生成。
下面是我们利用模型进行推理的一种实现方式，将希望简化的文本tokenized后传入模型，得到经过tokenizer解码后即可获得摘要后的文本。当然，读者也可以利用自己熟悉的方式进行生成。

In [28]:
def preprocess(items):
    inputs = []
    titles = []
    for item in items:
        inputs.append(item["abst"])
        titles.append(item["title"])
    return inputs, titles

In [29]:
test = read_json("csl/v1/test.json")
inputs, titles = preprocess(test)

Reading...: 100%|██████████| 500/500 [00:00<00:00, 116859.02it/s]


In [8]:
best_model = "test/best"
tokenizer = T5Tokenizer.from_pretrained(best_model)
model = T5ForConditionalGeneration.from_pretrained(best_model).cuda()

In [24]:
def predict(sources, batch_size=8):
    _model = model.eval() # 将模型转换为预测模式，使模型内容的droput失效。
    
    kwargs = {"num_beams":4}
    
    outputs = []
    for start in tqdm(range(0, len(sources), batch_size)):
        batch = sources[start:start+batch_size]
        
        input_tensor = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=512).input_ids.cuda()
        
        outputs.extend(model.generate(input_ids=input_tensor, **kwargs))
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [25]:
generations = predict(inputs)

100%|██████████| 63/63 [00:23<00:00,  2.74it/s]


In [27]:
generations[0]

'基于中心网络的实时数据多播应用'

In [30]:
titles[0]

'网络编码在实时战术数据多播中的应用'

#### 生成结果的评测

采用自动文摘任务上常用的自动评测指标Rouge-1, Rouge-2, Rouge-L对生成文本的质量进行评测。

In [41]:
from rouge import Rouge
rouge = Rouge()

def rouge_score(candidate, reference):
    text1 = " ".join(list(candidate))
    text2 = " ".join(list(reference))
    score = rouge.get_scores(text1, text2)
    print(score)
    return score

def compute_rouge(preds, refs):
    r1=[]
    r2=[]
    R_L=[]
    for pred, ref in zip(preds, refs):
        score = rouge_score(pred, ref)
        r1.append(scores[0]["rouge-1"]["f"])
        r2.append(scores[0]["rouge-2"]["f"])
        R_L.append(scores[0]["rouge-l"]["f"])
    return sum(r1)/len(r1), sum(r2)/len(r2), sum(R_L)/len(R_L)

In [ ]:
R_1, R_2, R_L = compute_rouge(generations, titles)

In [ ]:
R_L